<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/Pytorch015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

1. 输入嵌入

    输入嵌入将输入序列中的每个token转换为一个向量。这可以通过使用预训练的词向量（如Word2Vec或GloVe）或从头开始训练嵌入层来实现。


In [2]:
import torch.nn as nn

class InputEmbeddings(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        # 将词嵌入向量乘以根号d_model，以缩小数值范围，这有助于在训练过程中稳定梯度。
        return self.embedding(x) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))

In [3]:
import torch
import torch.nn as nn

class InputEmbeddings(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        # 将词嵌入向量乘以根号d_model，以缩小数值范围，这有助于在训练过程中稳定梯度。
        return self.embedding(x) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))

# 1. 定义超参数
d_model = 512  # 嵌入向量的维度
vocab_size = 10000  # 词汇表大小

# 2. 创建 InputEmbeddings 实例
input_embeddings = InputEmbeddings(d_model, vocab_size)

# 3. 创建一个随机输入张量
# 假设我们有一个形状为 (batch_size, seq_len) 的输入序列，其中每个元素都是词汇表中的一个索引。
batch_size = 32  # 批大小
seq_len = 64  # 序列长度
input_tensor = torch.randint(0, vocab_size, (batch_size, seq_len))  # 生成 [0, vocab_size) 之间的随机整数

# 4. 将输入张量传递给 InputEmbeddings 实例
output_tensor = input_embeddings(input_tensor)

# 5. 打印输入和输出的形状
print("Input tensor shape:", input_tensor.shape)  # 输出：torch.Size([32, 64])
print("Output tensor shape:", output_tensor.shape)  # 输出：torch.Size([32, 64, 512])

# 6. 打印输入张量的内容
print("Input tensor:", input_tensor)

# 7. 打印输出张量的内容
print("Output tensor:", output_tensor)


Input tensor shape: torch.Size([32, 64])
Output tensor shape: torch.Size([32, 64, 512])
Input tensor: tensor([[8297, 8897, 6965,  ..., 3620, 6531,   93],
        [1742, 2056, 1572,  ..., 5202, 4388,  615],
        [6934, 6253, 7875,  ..., 1671, 1969, 7301],
        ...,
        [5477, 3773, 3561,  ..., 6790, 6976, 7585],
        [4731, 3778, 7213,  ..., 8537, 1309, 7942],
        [4134, 1730, 8429,  ..., 7028, 1556, 8555]])
Output tensor: tensor([[[ 22.1121, -52.7231, -19.3767,  ...,  -3.3033,  15.5561,  32.3309],
         [-12.7012,  47.2343,  -1.4278,  ...,  -3.6128, -53.0922, -12.4165],
         [ 16.6640,  -7.8519,  55.3293,  ...,  21.8677,  10.0091, -11.9508],
         ...,
         [ -7.4874,  33.0481, -14.0697,  ..., -23.7683,  -1.8711, -10.5100],
         [ -9.2349, -20.4624,  19.2936,  ...,  16.5013, -23.3660, -24.7568],
         [-21.7927,  39.1972,   4.7612,  ..., -16.4555, -16.7975, -14.8032]],

        [[ -0.3361, -25.8222,   3.0963,  ...,  37.2056, -43.5989, -23.8237],
  


这段代码首先定义了 InputEmbeddings 类，然后创建了一个实例。接着，它生成了一个随机的输入张量，模拟一个包含词汇表中索引的序列。最后，它将输入张量传递给 InputEmbeddings 实例，并打印输入和输出张量的形状和内容。

通过运行这段代码，你应该能看到输入张量和输出张量的形状和内容。输入张量的形状是 (batch_size, seq_len)，表示一个包含 batch_size 个序列的批次，每个序列的长度为 seq_len。输出张量的形状是 (batch_size, seq_len, d_model)，表示输入序列中的每个词都被转换成了一个 d_model 维的嵌入向量。


2. 位置编码
    
    由于Transformer模型没有循环神经网络（RNN）那样的循环结构，它无法捕捉序列中token的位置信息。为了解决这个问题，我们使用位置编码将每个token的位置信息添加到其嵌入向量中。



In [4]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        # 创建一个形状为 (max_len, d_model) 的矩阵，用于保存位置编码
        pe = torch.zeros(max_len, d_model)
        # 创建一个形状为 (max_len, 1) 的向量，包含位置信息
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        # 计算位置除以 (10000 ** (2i / d_model)) 的值，用于后续计算
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        # 计算偶数位置的正弦值
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算奇数位置的余弦值
        pe[:, 1::2] = torch.cos(position * div_term)
        # 将 pe 的形状变为 (1, max_len, d_model)，以便与嵌入向量相加
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 将输入 x 的词嵌入向量与对应位置的位置编码相加
        # x.shape: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :]

In [5]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        # 创建一个形状为 (max_len, d_model) 的矩阵，用于保存位置编码
        pe = torch.zeros(max_len, d_model)
        # 创建一个形状为 (max_len, 1) 的向量，包含位置信息
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        # 计算位置除以 (10000 ** (2i / d_model)) 的值，用于后续计算
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        # 计算偶数位置的正弦值
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算奇数位置的余弦值
        pe[:, 1::2] = torch.cos(position * div_term)
        # 将 pe 的形状变为 (1, max_len, d_model)，以便与嵌入向量相加
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 将输入 x 的词嵌入向量与对应位置的位置编码相加
        # x.shape: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :]

# 1. 定义超参数
d_model = 512  # 嵌入向量的维度
max_len = 100  # 最大序列长度
batch_size = 32
seq_len = 64

# 2. 创建 PositionalEncoding 实例
positional_encoding = PositionalEncoding(d_model, max_len)

# 3. 创建一个随机输入张量
# 假设我们有一个形状为 (batch_size, seq_len, d_model) 的输入张量，其中每个元素都是一个嵌入向量。
input_tensor = torch.randn(batch_size, seq_len, d_model)

# 4. 将输入张量传递给 PositionalEncoding 实例
output_tensor = positional_encoding(input_tensor)

# 5. 打印输入和输出的形状
print("Input tensor shape:", input_tensor.shape)
print("Output tensor shape:", output_tensor.shape)

# 6. 打印输入张量的内容
print("Input tensor:", input_tensor)

# 7. 打印输出张量的内容
print("Output tensor:", output_tensor)


Input tensor shape: torch.Size([32, 64, 512])
Output tensor shape: torch.Size([32, 64, 512])
Input tensor: tensor([[[ 0.3855, -0.7737, -0.9055,  ..., -0.4490,  1.1517,  2.8363],
         [ 0.3909,  0.6545, -0.7708,  ..., -0.2495,  0.1720,  0.1927],
         [ 0.4242,  0.2979, -0.3946,  ..., -0.6007,  1.1717,  0.4679],
         ...,
         [ 0.2149,  1.1373,  0.7938,  ..., -1.8803, -0.1359, -1.1605],
         [-1.3158, -1.1940,  0.8334,  ..., -0.0222,  0.4556,  0.0550],
         [-1.4073,  0.4470,  0.9207,  ...,  1.0289, -0.9649, -1.6743]],

        [[ 1.0479,  1.3762,  0.5304,  ...,  0.0620, -0.3535,  0.4402],
         [-0.4265,  1.0887, -0.0819,  ..., -3.7146,  0.1422,  0.4274],
         [ 0.8953,  0.4231,  0.9791,  ..., -1.9236,  0.3852,  1.7296],
         ...,
         [ 0.4766, -0.8314, -1.3835,  ...,  2.0222,  1.9621,  1.6243],
         [-1.1289, -0.0748,  0.4628,  ..., -0.5372, -0.0684, -1.7344],
         [-0.1339,  0.1877, -1.1375,  ...,  0.3897,  0.4767, -0.9352]],

        [

这段代码首先定义了 PositionalEncoding 类，然后创建了一个实例。接着，它生成了一个随机的输入张量，模拟一个包含嵌入向量的序列。最后，它将输入张量传递给 PositionalEncoding 实例，并打印输入和输出张量的形状和内容。

通过运行这段代码，你应该能看到输入张量和输出张量的形状和内容。输入张量的形状是 (batch_size, seq_len, d_model)，输出张量的形状也是 (batch_size, seq_len, d_model)。输出张量的内容是输入张量和位置编码的逐元素相加的结果。




Transformer模型中的自注意力机制本身无法感知序列的顺序，因此需要位置编码来注入位置信息。位置编码的设计需要满足两个关键需求：每个位置应有唯一的编码，并且能捕捉位置间的相对关系。以下是详细解释：

---

### **1. 为什么用位置除以一个数？**
位置编码公式中，使用位置除以不同频率的缩放因子（如`10000^(2i/d_model)`），目的是让不同维度对应不同频率的正弦/余弦函数：
- **高频维度（低i）**：分母较小，频率高，能捕捉近距离的位置变化。
- **低频维度（高i）**：分母较大，频率低，能捕捉远距离的位置依赖。

例如，当`d_model=512`时，不同维度的波长从`2π`到`2π*10000`，覆盖了从局部到全局的位置关系。

---

### **2. 为什么分奇偶位置并用sin和cos？**
在代码中，偶数维度用正弦函数，奇数维度用余弦函数。这是为了利用三角函数的性质：
- **相对位置的可学习性**：通过正弦和余弦的线性组合，模型能轻松学到相对位置关系。例如，位置`pos+k`的编码可通过`pos`的编码线性变换得到。
- **唯一性与正交性**：正弦和余弦函数在不同频率下具有正交性，确保每个位置的编码唯一，同时避免不同维度间的信息冗余。

公式形式化如下：
$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right), \quad
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
$$

---

### **3. 代码实现解析**
```python
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        
        # 计算频率缩放因子：10000^(2i/d_model)的倒数
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() *
            (-math.log(10000.0) / d_model)
        )
        
        # 交替填充sin和cos
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数维度
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数维度
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]
```

- **`div_term`的作用**：通过指数和对数转换，高效计算频率缩放因子，避免直接计算大数的幂次。
- **奇偶交替填充**：相邻维度使用相同频率的正弦和余弦，增强位置关系的表达能力。

---

### **4. 位置编码的优势**
- **绝对位置**：每个位置有唯一编码。
- **相对位置**：通过三角恒等式（如$\sin(a+b)=\sin a \cos b + \cos a \sin b$），模型能学习到位置间的偏移。
- **泛化性**：固定的编码方式使模型能处理超出训练时最大长度的序列。

---

### **总结**
位置编码通过不同频率的正弦/余弦函数，为序列中的每个位置生成独特的嵌入。奇偶维度的交替设计利用三角函数的线性性质，使模型能同时捕捉绝对位置和相对位置信息，而频率缩放则让不同维度关注不同范围的位置关系。这种设计兼顾了效率与表达能力，是Transformer成功的关键之一。

3. 自注意力机制

    自注意力机制允许模型计算序列中不同位置之间的关系。对于序列中的每个token，自注意力机制会计算其与其他所有token之间的注意力权重，并使用这些权重来加权求和这些token的向量表示。


    * 没有自注意力机制，这个词就是“孤立化”的表示，独立于语境，与周围词没建立联系

    * 有了自注意力机制，这个词就是“社会化”的表示，融入到语境，与周围词建立了联系


In [6]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个head的维度

        # 定义用于计算query, key, value的线性层
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.W_O = nn.Linear(d_model, d_model)  # 输出线性层

    def forward(self, Q, K, V, mask=None):
        """
        计算自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)  # 用于屏蔽padding和未来信息
        Returns:
            attn_output: 注意力输出, shape (batch_size, seq_len, d_model)
        """
        batch_size = Q.size(0)
        seq_len = Q.size(1)

        # 1. 线性变换得到Q, K, V
        Q = self.W_Q(Q)  # (batch_size, seq_len, d_model)
        K = self.W_K(K)  # (batch_size, seq_len, d_model)
        V = self.W_V(V)  # (batch_size, seq_len, d_model)

        # 2. 将Q, K, V按head分割
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        K = K.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        V = V.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        # 3. 计算注意力分数
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))  # (batch_size, n_heads, seq_len, seq_len)

        # 4. 应用mask
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)  # 将mask为0的位置的score设为负无穷

        # 5. 计算注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch_size, n_heads, seq_len, seq_len)

        # 6. 计算加权后的V
        attn_output = torch.matmul(attn_weights, V)  # (batch_size, n_heads, seq_len, d_k)

        # 7. 拼接多头结果
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)  # (batch_size, seq_len, d_model)

        # 8. 通过线性层得到最终输出
        attn_output = self.W_O(attn_output)  # (batch_size, seq_len, d_model)

        return attn_output

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个head的维度

        # 定义用于计算query, key, value的线性层
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.W_O = nn.Linear(d_model, d_model)  # 输出线性层

    def forward(self, Q, K, V, mask=None):
        """
        计算自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)  # 用于屏蔽padding和未来信息
        Returns:
            attn_output: 注意力输出, shape (batch_size, seq_len, d_model)
        """
        batch_size = Q.size(0)
        seq_len = Q.size(1)

        # 1. 线性变换得到Q, K, V
        Q = self.W_Q(Q)  # (batch_size, seq_len, d_model)
        K = self.W_K(K)  # (batch_size, seq_len, d_model)
        V = self.W_V(V)  # (batch_size, seq_len, d_model)

        # 2. 将Q, K, V按head分割
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        K = K.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        V = V.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        # 3. 计算注意力分数
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))  # (batch_size, n_heads, seq_len, seq_len)

        # 4. 应用mask
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)  # 将mask为0的位置的score设为负无穷

        # 5. 计算注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch_size, n_heads, seq_len, seq_len)

        # 6. 计算加权后的V
        attn_output = torch.matmul(attn_weights, V)  # (batch_size, n_heads, seq_len, d_k)

        # 7. 拼接多头结果
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)  # (batch_size, seq_len, d_model)

        # 8. 通过线性层得到最终输出
        attn_output = self.W_O(attn_output)  # (batch_size, seq_len, d_model)

        return attn_output

# 1. 定义超参数
d_model = 512  # 嵌入向量的维度
n_heads = 8  # 注意力头的数量
batch_size = 32
seq_len = 64

# 2. 创建 SelfAttention 实例
self_attention = SelfAttention(d_model, n_heads)

# 3. 创建随机输入张量 Q, K, V
# 假设我们有形状为 (batch_size, seq_len, d_model) 的输入张量
Q = torch.randn(batch_size, seq_len, d_model)
K = torch.randn(batch_size, seq_len, d_model)
V = torch.randn(batch_size, seq_len, d_model)

# 4. 创建一个随机mask
# mask 的形状应该是 (batch_size, 1, seq_len, seq_len)
mask = torch.randint(0, 2, (batch_size, 1, seq_len, seq_len)).bool()  # 生成随机的布尔mask

# 5. 将输入张量传递给 SelfAttention 实例
output_tensor = self_attention(Q, K, V, mask)

# 6. 打印输入和输出的形状
print("Q shape:", Q.shape)
print("K shape:", K.shape)
print("V shape:", V.shape)
print("Mask shape:", mask.shape)
print("Output tensor shape:", output_tensor.shape)

# 7. 打印输入张量的内容
print("Q:", Q)
print("K:", K)
print("V:", V)
print("Mask:", mask)

# 8. 打印输出张量的内容
print("Output tensor:", output_tensor)


Q shape: torch.Size([32, 64, 512])
K shape: torch.Size([32, 64, 512])
V shape: torch.Size([32, 64, 512])
Mask shape: torch.Size([32, 1, 64, 64])
Output tensor shape: torch.Size([32, 64, 512])
Q: tensor([[[ 1.3543e+00,  2.1051e+00,  9.9099e-01,  ...,  8.9917e-01,
           7.3496e-02, -2.0353e-01],
         [ 1.0431e-01,  4.5047e-01, -3.2936e-01,  ...,  1.6953e-01,
           4.1585e-01,  4.9047e-01],
         [-5.7413e-01,  3.0500e-01,  2.2931e-01,  ...,  1.5950e-01,
          -1.6138e+00, -4.5554e-01],
         ...,
         [-9.9080e-01,  1.2162e+00,  5.1583e-02,  ...,  2.9905e-01,
          -9.5179e-01, -3.0786e-01],
         [ 5.8014e-01, -1.2049e+00,  1.7700e-01,  ..., -1.9418e+00,
          -1.5116e+00, -1.5093e-02],
         [-1.1372e+00,  1.7667e+00,  1.3713e+00,  ..., -8.5653e-03,
           1.5630e+00,  6.8038e-01]],

        [[-5.0720e-01,  4.6156e-01,  6.4562e-01,  ..., -2.7373e-02,
          -1.4242e+00,  4.2019e-01],
         [ 1.0535e+00, -6.9789e-01,  1.4447e+00,  ...,

### **自注意力机制：让词语学会“社交”的魔法**

---

#### **1. 直观理解：从“孤立”到“社交”**
想象你在一场聚会中：
- **没有自注意力**：每个人只能自言自语，不知道周围人在说什么。就像词语被孤立，每个词只能表达自己的意思，无法理解上下文。
- **有自注意力**：每个人开始观察周围人的表情和动作，调整自己的发言。词语之间开始“交流”，每个词的含义会根据上下文动态变化。

**例如**：
- **孤立**：句子 *"苹果股价上涨了"* 中的“苹果”可能指水果或公司。
- **社交**：通过自注意力，“苹果”会关注“股价”和“上涨”，从而确定自己指代公司。

---

### **2. 自注意力机制的核心思想**
自注意力机制让每个词主动做三件事：
1. **提问（Query）**：当前词提出一个问题（例如：“我应该关注哪些词？”）
2. **回答（Key）**：所有词给出答案（例如：“我的关键词是什么？”）
3. **整合（Value）**：根据问题和答案的匹配程度，汇总所有词的信息

---

### **3. 代码分步解析**

```python
class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model  # 输入维度（如512）
        self.n_heads = n_heads  # 多头注意力头数（如8）
        self.d_k = d_model // n_heads  # 每头的维度（如64）

        # 定义线性变换矩阵
        self.W_Q = nn.Linear(d_model, d_model)  # 生成Query
        self.W_K = nn.Linear(d_model, d_model)  # 生成Key
        self.W_V = nn.Linear(d_model, d_model)  # 生成Value
        self.W_O = nn.Linear(d_model, d_model)  # 输出变换
```

---

#### **步骤 1：生成Q/K/V（提问、回答、内容）**
- **输入**：每个词的嵌入向量（含语义信息）
- **操作**：通过线性变换生成三个角色
  ```python
  Q = self.W_Q(x)  # Query：当前词提出的问题
  K = self.W_K(x)  # Key：其他词提供的答案关键词
  V = self.W_V(x)  # Value：其他词的实际内容
  ```
- **意义**：让每个词既能主动提问，也能被动回答问题。

---

#### **步骤 2：多头分割（多视角观察）**
```python
Q = Q.view(batch_size, seq_len, n_heads, d_k).transpose(1, 2)
```
- **原理**：将向量拆分为多个“头”（类似多组独立的眼睛），每组关注不同的语义关系。
- **示例**：
  - 头1关注“语法关系”（如主谓宾）
  - 头2关注“语义关系”（如同义词）
  - 头3关注“指代关系”（如代词指向）

---

#### **步骤 3：计算注意力分数（社交匹配度）**
```python
attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / sqrt(d_k)
```
- **数学意义**：通过点积计算Query和Key的相似度。
- **直观解释**：
  - 点积越大 → 问题与答案越相关
  - 除以`sqrt(d_k)`：防止维度过高导致点积过大，让梯度更稳定

---

#### **步骤 4：Softmax归一化（分配注意力权重）**
```python
attn_weights = F.softmax(attn_scores, dim=-1)
```
- **作用**：将分数转化为概率分布，权重总和为1。
- **示例**：
  ```
  "猫吃鱼"中，“吃”的注意力权重可能是：
  猫: 0.7, 吃: 0.1, 鱼: 0.2
  ```

---

#### **步骤 5：加权聚合（信息整合）**
```python
attn_output = torch.matmul(attn_weights, V)
```
- **意义**：用注意力权重对Value加权求和，得到当前词的新表示。
- **效果**：
  - 若“苹果”关注“股价”→ 新表示包含公司信息
  - 若“苹果”关注“红色”→ 新表示包含水果信息

---

#### **步骤 6：多头拼接（综合多视角信息）**
```python
attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
```
- **原理**：将多个头的结果拼接，恢复原始维度，综合不同视角的信息。

---

### **4. 自注意力的独特优势**
| 特性                | 说明                          | 类比                      |
|---------------------|-----------------------------|-------------------------|
| **动态权重**          | 每个位置的关注权重实时计算          | 实时调整社交圈               |
| **并行计算**          | 所有位置的关系可同时计算            | 多人同时交谈                 |
| **长距离依赖**        | 直接捕捉任意距离的词语关系          | 跨房间对话                  |
| **可解释性**          | 注意力权重可视化显示词语关联        | 社交关系图谱                |

---

### **5. 对比传统方法**
|                  | RNN                        | CNN                | Self-Attention         |
|------------------|----------------------------|--------------------|------------------------|
| **计算方式**      | 顺序计算（依赖前一时刻）          | 局部窗口滑动         | 全连接并行计算            |
| **长距离依赖**    | 容易衰减（梯度消失）             | 需要多层堆叠         | 直接捕捉（一步计算）        |
| **可并行性**      | 差                         | 中等               | 优                     |
| **示例**         | “I am from China”→处理到“China”时可能已忘记“I” | 只能看到局部词语组合 | 让“China”直接关注“I”    |

---

### **6. 终极总结**
自注意力机制通过三个关键设计：
1. **Q/K/V角色分离**：实现“提问-回答-整合”的社交逻辑
2. **多头机制**：多视角观察上下文关系
3. **动态权重**：根据语境灵活调整关注重点

让每个词语从“孤立个体”变成“社交达人”，最终使模型理解：
- *“bank”在“河边”和“金融”的不同含义*
- *“他”指的是前文提到的哪个名词*
- *“虽然...但是...”之间的转折关系*

这正是Transformer模型超越传统方法的核心魔法！ 🧙♂️


4. 多头注意力机制

    多头注意力机制是自注意力机制的扩展，它允许模型同时关注序列中不同的关系。具体来说，多头注意力机制将输入序列分成多个“头”，并在每个头上独立计算自注意力，然后将所有头的结果拼接起来。



### **多头拼接：像“侦探团队破案”一样整合信息**

---

#### **1. 通俗比喻：侦探团队协作**
想象你是一个侦探队长，调查一起复杂案件：
- **单头注意力**：你独自分析所有线索，可能因为视角单一忽略关键细节。
- **多头注意力**：你组建了一个侦探小队，每个侦探专注不同方向：
  - **侦探A**：研究时间线（关注词语出现的顺序）
  - **侦探B**：分析人物关系（关注代词指向）
  - **侦探C**：观察情感倾向（关注褒贬义词）
  - **侦探D**：梳理逻辑结构（关注连词和转折词）

**最终破案**：每个侦探提交自己的调查报告，你把这些报告的关键结论拼接起来，得到完整真相。

---

#### **2. 代码拆解：如何实现“侦探团队协作”**
```python
# 假设输入向量维度d_model=512，头数n_heads=8
# 每个头的维度d_k = 512 / 8 = 64

# 步骤1：分割多头（组建侦探团队）
Q = Q.view(batch_size, seq_len, 8, 64).transpose(1, 2)  # 形状变为 (batch_size, 8, seq_len, 64)
# 现在Q被拆分成8个侦探（头），每个侦探携带64维的观察工具

# 步骤2：各头独立工作（侦探各自调查）
attn_output = ...  # 每个头独立计算得到 (batch_size, 8, seq_len, 64)

# 步骤3：拼接结果（汇总调查报告）
attn_output = attn_output.transpose(1, 2)  # 形状变为 (batch_size, seq_len, 8, 64)
attn_output = attn_output.contiguous().view(batch_size, seq_len, 512)  # 拼接后恢复512维
```

---

#### **3. 关键设计：为什么不是简单相加？**
假设每个侦探的调查报告是64页纸：
- **直接相加**：把8份64页的报告叠加，会得到一堆混乱的重复信息。
- **拼接后线性变换**：把8份报告首尾相连（共8×64=512页），然后用一个智能文件柜（`W_O`线性层）重新整理：
  - 合并相似结论（如多个侦探都提到时间矛盾）
  - 过滤冗余信息（如重复的现场描述）
  - 突出关键线索（如某个侦探发现的指纹细节）

---

#### **4. 实例解析：处理句子 "他打了那个骂他朋友的人"**
- **侦探A（语法分析头）**：
  - 发现“打”是动词，关注施动者“他”和受动者“人”
  
- **侦探B（指代分析头）**：
  - 确定第一个“他”和第二个“他”指同一人
  - “朋友”属于第一个“他”

- **侦探C（逻辑关系头）**：
  - 分析“骂”是“打”的原因
  
- **拼接后**：
  ```python
  # 每个头输出的64维向量可能包含：
  [侦探A的语法结论] + [侦探B的指代结论] + [侦探C的逻辑结论] + ...其他侦探结论
  ```
  - 最终通过`W_O`线性层，模型理解：“他因为朋友被骂而打人”

---

#### **5. 多头机制的三大优势**
| 优势                | 解释                          | 类比                      |
|---------------------|-----------------------------|-------------------------|
| **信息多样性**       | 不同头捕获不同类型的语义关系        | 多学科专家会诊              |
| **抗干扰能力**       | 即使某个头失效，其他头仍可补救       | 团队中有人犯错不影响最终结论     |
| **高效并行**         | 各头独立计算，充分利用GPU并行能力    | 侦探们同时调查不同线索          |

---

#### **6. 和单头注意力的直观对比**
- **单头效果**：
  ```python
  # 假设所有头合并成一个512维的大头
  attn_output = [单一侦探的512页冗长报告]
  ```
  - 容易陷入局部观察（例如只关注语法忽略情感）
  - 处理复杂句子时准确率下降

- **多头效果**：
  ```python
  # 8个头各司其职
  attn_output = [语法报告64页] + [指代报告64页] + [情感报告64页] + ...
  ```
  - 各领域专家分工协作
  - 对歧义句子的处理更精准（如“我喜欢苹果”中识别是水果还是品牌）

---

#### **7. 可视化理解：拼接就像乐高积木**
- **每个头**：一块特定形状的乐高积木（如齿轮、车轮、窗户）
- **拼接**：把不同形状的积木首尾相连，组合成一辆完整的乐高汽车
- **线性层（`W_O`）**：调整积木位置和角度，让车轮在底部、窗户在顶部，最终成为可行驶的汽车

---

### **终极总结**
多头拼接的精髓在于：
1. **分工**：让不同的“子专家”（头）关注不同维度的信息
2. **协作**：通过拼接和线性变换整合多视角观察结果
3. **升华**：最终输出比任何单头更全面、更准确的语义表示

就像通过多棱镜观察白光会分解出七彩颜色，多头机制让模型从多个角度“折射”语义，最终合成更丰富的理解！ 🌈

In [8]:

import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.self_attention = SelfAttention(d_model, n_heads)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, mask=None):
        """
        计算多头自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 多头自注意力的输出, shape (batch_size, seq_len, d_model)
        """
        # 计算自注意力
        attn_output = self.self_attention(Q, K, V, mask)

        # dropout + 残差连接 + layer norm
        output = self.dropout(attn_output) + Q
        output = self.layer_norm(output)

        return output


In [9]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个head的维度

        # 定义用于计算query, key, value的线性层
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.W_O = nn.Linear(d_model, d_model)  # 输出线性层

    def forward(self, Q, K, V, mask=None):
        """
        计算自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)  # 用于屏蔽padding和未来信息
        Returns:
            attn_output: 注意力输出, shape (batch_size, seq_len, d_model)
        """
        batch_size = Q.size(0)
        seq_len = Q.size(1)

        # 1. 线性变换得到Q, K, V
        Q = self.W_Q(Q)  # (batch_size, seq_len, d_model)
        K = self.W_K(K)  # (batch_size, seq_len, d_model)
        V = self.W_V(V)  # (batch_size, seq_len, d_model)

        # 2. 将Q, K, V按head分割
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        K = K.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        V = V.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        # 3. 计算注意力分数
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))  # (batch_size, n_heads, seq_len, seq_len)

        # 4. 应用mask
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)  # 将mask为0的位置的score设为负无穷

        # 5. 计算注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch_size, n_heads, seq_len, seq_len)

        # 6. 计算加权后的V
        attn_output = torch.matmul(attn_weights, V)  # (batch_size, n_heads, seq_len, d_k)

        # 7. 拼接多头结果
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)  # (batch_size, seq_len, d_model)

        # 8. 通过线性层得到最终输出
        attn_output = self.W_O(attn_output)  # (batch_size, seq_len, d_model)

        return attn_output

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.self_attention = SelfAttention(d_model, n_heads)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, mask=None):
        """
        计算多头自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 多头自注意力的输出, shape (batch_size, seq_len, d_model)
        """
        # 计算自注意力
        attn_output = self.self_attention(Q, K, V, mask)

        # dropout + 残差连接 + layer norm
        output = self.dropout(attn_output) + Q
        output = self.layer_norm(output)

        return output

# 1. 定义超参数
d_model = 512  # 嵌入向量的维度
n_heads = 8  # 注意力头的数量
batch_size = 32
seq_len = 64

# 2. 创建 MultiHeadAttention 实例
multi_head_attention = MultiHeadAttention(d_model, n_heads)

# 3. 创建随机输入张量 Q, K, V
# 假设我们有形状为 (batch_size, seq_len, d_model) 的输入张量
Q = torch.randn(batch_size, seq_len, d_model)
K = torch.randn(batch_size, seq_len, d_model)
V = torch.randn(batch_size, seq_len, d_model)
mask = torch.randint(0, 2, (batch_size, 1, seq_len, seq_len)).bool()  # 生成随机的布尔mask

# 4. 将输入张量传递给 MultiHeadAttention 实例
output_tensor = multi_head_attention(Q, K, V, mask)

# 5. 打印输入和输出的形状
print("Q shape:", Q.shape)
print("K shape:", K.shape)
print("V shape:", V.shape)
print("Mask shape:", mask.shape)
print("Output tensor shape:", output_tensor.shape)

# 6. 打印输入张量的内容
print("Q:", Q)
print("K:", K)
print("V:", V)
print("Mask:", mask)

# 7. 打印输出张量的内容
print("Output tensor:", output_tensor)


Q shape: torch.Size([32, 64, 512])
K shape: torch.Size([32, 64, 512])
V shape: torch.Size([32, 64, 512])
Mask shape: torch.Size([32, 1, 64, 64])
Output tensor shape: torch.Size([32, 64, 512])
Q: tensor([[[ 4.0043e-01,  1.2000e-01, -1.2407e+00,  ...,  8.2515e-01,
           1.7532e+00,  1.9152e-01],
         [-5.8081e-01,  9.4857e-01, -9.8336e-01,  ..., -1.5724e-01,
           4.0996e-01, -7.3585e-01],
         [-2.5012e-01,  1.5524e-01, -1.0508e+00,  ..., -2.0908e+00,
          -3.1061e-01, -1.6027e+00],
         ...,
         [-1.9709e+00,  4.1245e-01, -2.2308e-01,  ...,  4.2023e-01,
           1.8654e+00, -1.3326e-01],
         [-9.2165e-02,  9.5081e-01,  7.0895e-01,  ..., -2.7355e-01,
           5.6614e-01, -1.2911e+00],
         [ 1.2471e+00,  3.7250e-01, -2.5672e+00,  ..., -2.6930e+00,
          -3.6622e-01,  1.1147e-01]],

        [[ 8.2169e-01,  1.1999e+00, -1.8431e+00,  ..., -1.1390e-01,
          -3.1040e-01, -5.0482e-01],
         [-9.4894e-01,  1.0505e+00,  5.8862e-01,  ...,


这段代码首先定义了 MultiHeadAttention 类，然后创建了一个实例。接着，它生成了随机的输入张量 Q, K, 和 V，以及一个随机的 mask。最后，它将这些张量传递给 MultiHeadAttention 实例，并打印输入和输出张量的形状和内容。

通过运行这段代码，你应该能看到输入张量 Q, K, 和 V 的形状和内容，以及输出张量的形状和内容。所有输入张量的形状都是 (batch_size, seq_len, d_model)，输出张量的形状也是 (batch_size, seq_len, d_model)。



5. 前馈神经网络

    前馈神经网络（FFN）用于对每个位置的向量进行非线性变换。它由两个线性层组成，中间有一个ReLU激活函数。


In [10]:
import torch.nn as nn

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        前馈神经网络

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        # 线性层 + ReLU + 线性层
        output = self.linear_2(self.relu(self.linear_1(x)))
        output = self.dropout(output) + x  # dropout + 残差连接
        output = self.layer_norm(output)  # layer norm
        return output

In [11]:
import torch
import torch.nn as nn

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        前馈神经网络

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        # 线性层 + ReLU + 线性层
        output = self.linear_2(self.relu(self.linear_1(x)))
        output = self.dropout(output) + x  # dropout + 残差连接
        output = self.layer_norm(output)  # layer norm
        return output

# 1. 定义超参数
d_model = 512  # 嵌入向量的维度
d_ff = 2048  # 前馈神经网络的隐藏层维度
batch_size = 32
seq_len = 64

# 2. 创建 FeedForward 实例
feed_forward = FeedForward(d_model, d_ff)

# 3. 创建一个随机输入张量
# 假设我们有一个形状为 (batch_size, seq_len, d_model) 的输入张量
input_tensor = torch.randn(batch_size, seq_len, d_model)

# 4. 将输入张量传递给 FeedForward 实例
output_tensor = feed_forward(input_tensor)

# 5. 打印输入和输出的形状
print("Input tensor shape:", input_tensor.shape)
print("Output tensor shape:", output_tensor.shape)

# 6. 打印输入张量的内容
print("Input tensor:", input_tensor)

# 7. 打印输出张量的内容
print("Output tensor:", output_tensor)


Input tensor shape: torch.Size([32, 64, 512])
Output tensor shape: torch.Size([32, 64, 512])
Input tensor: tensor([[[ 0.1017,  1.6102, -1.0562,  ..., -0.9411, -1.3324, -0.1248],
         [-0.1562, -0.4736, -0.1512,  ..., -0.0599, -1.6264, -0.1323],
         [ 0.7724,  1.8818, -0.3402,  ..., -2.8729,  1.4256, -0.1110],
         ...,
         [ 0.8633,  0.4225, -1.0925,  ..., -1.1562, -0.8201,  0.5502],
         [ 1.4233, -0.9024, -1.3541,  ..., -0.3882,  0.9496, -0.6593],
         [ 0.1632, -0.6885,  0.6971,  ...,  1.1138, -0.6352, -0.4801]],

        [[-0.6802, -0.2314,  0.2467,  ..., -0.1419, -1.1976,  0.0106],
         [-0.7010, -1.3776, -0.8094,  ..., -1.0240, -0.3757, -1.0994],
         [ 1.2871, -0.5674,  0.0573,  ..., -0.0879,  0.7120, -2.0591],
         ...,
         [ 0.3913, -0.4572,  0.5571,  ..., -0.9398,  0.3013,  0.8204],
         [ 0.3829, -1.7627, -0.5273,  ..., -1.4363, -0.4933,  1.3425],
         [ 0.4029,  1.1732, -0.6661,  ...,  0.2858, -0.5831,  0.2284]],

        [


这段代码首先定义了 FeedForward 类，然后创建了一个实例。接着，它生成了一个随机的输入张量，模拟一个包含嵌入向量的序列。最后，它将输入张量传递给 FeedForward 实例，并打印输入和输出张量的形状和内容。

通过运行这段代码，你应该能看到输入张量和输出张量的形状和内容。输入张量的形状是 (batch_size, seq_len, d_model)，输出张量的形状也是 (batch_size, seq_len, d_model)。输出张量的内容是输入张量经过前馈神经网络处理后的结果。



### **FeedForward与残差连接：像“滚雪球”一样积累知识**

---

#### **1. 通俗比喻：知识迭代升级**
想象你在写一篇论文：
- **无残差连接**：每修改一版都要从头重写，之前的思考全部丢弃。
- **有残差连接**：在上一版基础上修改，保留有价值的内容，只调整需要改进的部分。

**例如**：
- 初稿：*“Transformer是一种强大的模型”*
- 修改稿（残差思想）：*“Transformer是一种通过自注意力机制实现长距离依赖捕捉的强大模型”*

---

#### **2. 代码拆解：知识如何“滚雪球”**
```python
class FeedForward(nn.Module):
    def forward(self, x):
        # 原始输入x：当前知识状态
        new_knowledge = self.linear_2(self.relu(self.linear_1(x)))  # 学习新知识
        output = self.dropout(new_knowledge) + x  # 新知识叠加到旧知识上（残差连接）
        output = self.layer_norm(output)  # 知识标准化
        return output
```

---

#### **3. 残差连接的四大核心作用**
| 作用                | 解释                          | 类比                      |
|---------------------|-----------------------------|-------------------------|
| **防知识丢失**       | 确保底层重要信息不被覆盖           | 论文保留历史版本             |
| **梯度高速公路**     | 允许梯度直接回流到底层             | 建立作者-审稿人的直达沟通通道    |
| **渐进式学习**       | 每次只学习需要调整的部分            | 逐步完善论文而非推翻重写        |
| **抗过拟合**         | Dropout只作用于新增部分，保留主干   | 修改时只调整局部，保持整体稳定性   |

---

#### **4. 实例解析：处理句子 "Deep Learning is fascinating"**
- **输入x**：经过自注意力后的词向量
- **FeedForward过程**：
  1. 线性层1：提取高阶特征（如将"fascinating"关联到"neural networks"）
  2. ReLU：过滤负值（保留关键特征）
  3. 线性层2：降维恢复原始维度
  4. **残差相加**：保留自注意力层捕捉的词语关系，叠加FFN提取的语义特征
  5. LayerNorm：平衡特征尺度

**最终效果**：
- 既保留了*自注意力层发现的"Deep Learning"与"fascinating"的关系*
- 又增加了*FFN层理解的"fascinating"隐含的"complex mathematical models"信息*

---

#### **5. 残差连接 vs 普通连接**
| 场景               | 普通连接                         | 残差连接                          |
|--------------------|--------------------------------|---------------------------------|
| **梯度传播**        | 梯度逐层衰减，底层更新困难             | 梯度可直达底层，缓解消失问题            |
| **深层网络表现**    | 超过20层后准确率下降                 | 100+层仍保持稳定（如GPT-3有96层）       |
| **信息保留**        | 逐层覆盖原始信息                    | 像版本控制系统保留所有重要修改           |
| **物理意义**        | 强制模型学习绝对变换                 | 让模型专注学习相对调整（残差）           |

---

#### **6. 可视化理解：残差如同“传送带”**
- **传统网络**：货物（数据）每经过一个加工站（网络层），都要完全重新包装
  ![](https://example.com/normal_network.gif)
- **残差网络**：传送带直接贯通所有加工站，工人（网络参数）只需在传送带上添加新零件
  ![](https://example.com/residual_network.gif)

---

### **终极总结**
残差连接的精妙之处在于：
1. **加法设计**：`output = F(x) + x` 让网络只需学习差值
2. **常态保留**：原始信息始终在场，避免"一错到底"
3. **动态平衡**：LayerNorm在残差后实施，既规范输出又不压制原始信号

就像优秀的作家不断在原有稿子上精修，而不是每次重写，残差连接让神经网络得以在深层结构中稳定积累知识，最终成就了Transformer这类大模型的强大能力！ ✨

6. 编码器层

    编码器层是Transformer编码器的基本组成部分。它由一个多头自注意力子层和一个前馈神经网络子层组成。


In [12]:
import torch.nn as nn

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, mask=None):
        """
        编码器层

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        output = self.multi_head_attention(x, x, x, mask)
        output = self.feed_forward(output)
        return output


7. 编码器

    编码器由多个相同的编码器层堆叠而成。它用于处理输入序列，并将其转换为一个中间表示，该中间表示将用于解码器生成输出序列。


In [13]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, mask=None):
        """
        编码器

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, mask)
        return x

In [14]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个head的维度

        # 定义用于计算query, key, value的线性层
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.W_O = nn.Linear(d_model, d_model)  # 输出线性层

    def forward(self, Q, K, V, mask=None):
        """
        计算自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)  # 用于屏蔽padding和未来信息
        Returns:
            attn_output: 注意力输出, shape (batch_size, seq_len, d_model)
        """
        batch_size = Q.size(0)
        seq_len = Q.size(1)

        # 1. 线性变换得到Q, K, V
        Q = self.W_Q(Q)  # (batch_size, seq_len, d_model)
        K = self.W_K(K)  # (batch_size, seq_len, d_model)
        V = self.W_V(V)  # (batch_size, seq_len, d_model)

        # 2. 将Q, K, V按head分割
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        K = K.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        V = V.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        # 3. 计算注意力分数
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))  # (batch_size, n_heads, seq_len, seq_len)

        # 4. 应用mask
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)  # 将mask为0的位置的score设为负无穷

        # 5. 计算注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch_size, n_heads, seq_len, seq_len)

        # 6. 计算加权后的V
        attn_output = torch.matmul(attn_weights, V)  # (batch_size, n_heads, seq_len, d_k)

        # 7. 拼接多头结果
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)  # (batch_size, seq_len, d_model)

        # 8. 通过线性层得到最终输出
        attn_output = self.W_O(attn_output)  # (batch_size, seq_len, d_model)

        return attn_output

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.self_attention = SelfAttention(d_model, n_heads)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, mask=None):
        """
        计算多头自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 多头自注意力的输出, shape (batch_size, seq_len, d_model)
        """
        # 计算自注意力
        attn_output = self.self_attention(Q, K, V, mask)

        # dropout + 残差连接 + layer norm
        output = self.dropout(attn_output) + Q
        output = self.layer_norm(output)

        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        前馈神经网络

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        # 线性层 + ReLU + 线性层
        output = self.linear_2(self.relu(self.linear_1(x)))
        output = self.dropout(output) + x  # dropout + 残差连接
        output = self.layer_norm(output)  # layer norm
        return output

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, mask=None):
        """
        编码器层

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        output = self.multi_head_attention(x, x, x, mask)
        output = self.feed_forward(output)
        return output

class Encoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, mask=None):
        """
        编码器

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, mask)
        return x

# 1. 定义超参数
n_layers = 6  # 编码器层数
d_model = 512  # 嵌入向量的维度
n_heads = 8  # 注意力头的数量
d_ff = 2048  # 前馈神经网络的隐藏层维度
batch_size = 32
seq_len = 64

# 2. 创建 Encoder 实例
encoder = Encoder(n_layers, d_model, n_heads, d_ff)

# 3. 创建一个随机输入张量
# 假设我们有一个形状为 (batch_size, seq_len, d_model) 的输入张量
input_tensor = torch.randn(batch_size, seq_len, d_model)

# 4. 创建一个随机mask
# mask 的形状应该是 (batch_size, 1, seq_len, seq_len)
mask = torch.randint(0, 2, (batch_size, 1, seq_len, seq_len)).bool()  # 生成随机的布尔mask

# 5. 将输入张量传递给 Encoder 实例
output_tensor = encoder(input_tensor, mask)

# 6. 打印输入和输出的形状
print("Input tensor shape:", input_tensor.shape)
print("Mask shape:", mask.shape)
print("Output tensor shape:", output_tensor.shape)

# 7. 打印输入张量的内容
print("Input tensor:", input_tensor)
print("Mask:", mask)

# 8. 打印输出张量的内容
print("Output tensor:", output_tensor)


Input tensor shape: torch.Size([32, 64, 512])
Mask shape: torch.Size([32, 1, 64, 64])
Output tensor shape: torch.Size([32, 64, 512])
Input tensor: tensor([[[-1.3209e+00,  4.9295e-01, -4.9741e-01,  ..., -4.9661e-01,
          -7.5814e-01, -3.2700e-01],
         [ 8.1219e-01, -1.4553e+00,  1.0710e-01,  ...,  1.0356e+00,
           4.2538e-01,  1.3326e+00],
         [-5.7732e-01, -2.0885e+00, -6.4024e-02,  ..., -2.8878e+00,
          -1.9811e-01,  9.6770e-01],
         ...,
         [ 1.4377e+00,  7.8260e-01,  1.3218e+00,  ..., -7.3485e-01,
          -6.6175e-01,  9.2765e-01],
         [ 1.5755e+00, -3.2009e-01, -3.8572e-01,  ...,  1.1086e+00,
           7.7100e-01,  1.7608e-01],
         [-5.9777e-01, -5.1152e-01, -2.3108e-01,  ...,  9.3014e-02,
           1.4436e+00,  2.6183e-01]],

        [[-2.8499e-01, -5.1307e-01,  1.7653e-01,  ..., -2.4935e-01,
           4.8257e-01, -8.5938e-01],
         [-6.4622e-01, -1.8943e-02, -3.1036e-01,  ..., -5.1551e-01,
           6.1017e-01,  1.2147e+00

这段代码首先定义了 Encoder 类及其依赖的 EncoderLayer、MultiHeadAttention 和 FeedForward 类。然后，它创建了一个 Encoder 实例。接着，它生成了一个随机的输入张量和一个随机的 mask。最后，它将输入张量和 mask 传递给 Encoder 实例，并打印输入和输出张量的形状和内容。

通过运行这段代码，你应该能看到输入张量、mask 和输出张量的形状和内容。输入张量的形状是 (batch_size, seq_len, d_model)，mask 的形状是 (batch_size, 1, seq_len, seq_len)，输出张量的形状也是 (batch_size, seq_len, d_model)。输出张量的内容是输入张量经过编码器处理后的结果。

### **Encoder：像“多层精炼工厂”一样理解文本**

---

#### **1. 直观比喻：文本理解的流水线**
想象一家矿石精炼厂的处理流程：
- **原始矿石**：输入的文本序列（如"I love natural language processing"）
- **第1层处理**：破碎矿石 → 识别基本语法结构（主谓宾）
- **第2层处理**：分离金属 → 捕捉语义关系（"processing"与"language"的修饰关系）
- **第3层处理**：提纯冶炼 → 理解深层含义（这句话表达对NLP领域的热情）
- **最终产品**：高度抽象的语义表示，可供下游任务（翻译/分类）直接使用

---

#### **2. 代码拆解：知识精炼流水线**
```python
class EncoderLayer(nn.Module):
    def forward(self, x, mask=None):
        # 第一车间：自注意力车间（建立全局关联）
        x = self.multi_head_attention(x, x, x, mask)  # 每个词与其他词对话
        
        # 第二车间：前馈神经网络车间（深度加工特征）
        x = self.feed_forward(x)  # 每个词独立思考
        
        return x  # 精炼后的特征

class Encoder(nn.Module):
    def forward(self, x, mask=None):
        # 经过N层精炼（典型6层）
        for layer in self.layers:
            x = layer(x, mask)  # 每层在前一层基础上继续加工
        return x
```

---

#### **3. Encoder的三大核心思想**
| 思想                | 解释                          | 类比                      |
|---------------------|-----------------------------|-------------------------|
| **渐进式抽象**       | 每层逐步提取更高阶的特征            | 绘画：线稿→底色→细节           |
| **参数共享架构**     | 所有层结构相同但参数独立              | 工厂每层设备相同但调节参数不同      |
| **信息无损传递**     | 残差连接确保底层信息直达高层           | 文件修订保留所有历史批注          |

---

#### **4. 逐层工作原理示例**
处理句子："The animal didn't cross the street because it was too tired"

- **第1层**：
  - 建立基础关联："it"初步关联到"animal"和"street"
  - 识别否定结构："didn't"与"cross"的关系

- **第3层**：
  - 确定指代关系："it"明确指向"animal"
  - 理解因果逻辑："because"连接结果与原因

- **第6层**：
  - 捕捉深层语义：整个句子的隐含重点是"动物的状态"
  - 准备适用于下游任务的特征表示

---

#### **5. 与CNN/RNN的对比**
|                  | CNN                        | RNN                | Transformer Encoder |
|------------------|----------------------------|--------------------|---------------------|
| **特征提取方式**  | 局部窗口卷积                 | 顺序递归处理         | 全局自注意力           |
| **长距离依赖**    | 需要多层堆叠                 | 容易遗忘早期信息      | 单层即可捕捉           |
| **并行性**        | 高                         | 低                 | 极高                 |
| **典型应用**      | 图像分类                    | 短文本生成           | 需要深度理解的任务       |

---

#### **6. 可视化理解：特征演变动画**
假设用颜色表示特征变化：
- **输入层**：每个词是纯色块（红/蓝/绿...）
- **第1层**：颜色开始混合（红+蓝→紫色表示动词关系）
- **第3层**：出现金属光泽（指代关系用反光效果表现）
- **输出层**：所有词呈现统一的金色（完成语义融合）

---

### **7. Encoder的独特设计哲学**
1. **横向对称性**：所有Encoder层结构完全相同
   - 优势：方便扩展（如BERT-base有12层，large有24层）
   - 实现：通过`nn.ModuleList`统一管理

2. **纵向差异性**：每层通过独立参数学习不同模式
   - 低层：关注语法/词性等基础特征
   - 高层：捕捉语义/逻辑等抽象特征

3. **稳定化设计**：
   - 残差连接：防止深层网络退化
   - LayerNorm：控制特征尺度
   - Dropout：增强泛化能力

---

### **终极总结**
Encoder就像一组智能滤网：
1. **多层筛滤**：每层过滤掉无关信息，保留关键特征
2. **逐步提纯**：从表层语法到深层语义的渐进理解
3. **稳定传导**：通过残差连接确保信息无损传递

这种设计使得Transformer能够像人类一样：
- 第一遍阅读抓主旨
- 第二遍阅读理逻辑
- 第三遍阅读悟深意

最终将原始文本转化为蕴含丰富语义的特征表示，为后续的解码器（Decoder）提供强大的理解基础！ 🚀


8. 解码器层

    解码器层是Transformer解码器的基本组成部分。它由三个子层组成：一个多头自注意力子层、一个编码器-解码器注意力子层和一个前馈神经网络子层。


In [15]:

import torch.nn as nn

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(d_model, n_heads)  # Masked Self Attention
        self.encoder_decoder_attention = MultiHeadAttention(d_model, n_heads)  # Encoder-Decoder Attention
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        解码器层

        Args:
            x: 输入, shape (batch_size, tgt_seq_len, d_model)
            encoder_output: 编码器的输出, shape (batch_size, src_seq_len, d_model)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, d_model)
        """
        # Masked self attention
        output = self.masked_multi_head_attention(x, x, x, tgt_mask)

        # Encoder-decoder attention
        output = self.encoder_decoder_attention(output, encoder_output, encoder_output, src_mask)

        # Feed forward
        output = self.feed_forward(output)
        return output



9. 解码器

    解码器由多个相同的解码器层堆叠而成。它用于生成输出序列，一次生成一个token。


In [16]:

import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        解码器

        Args:
            x: 输入, shape (batch_size, tgt_seq_len, d_model)
            encoder_output: 编码器的输出, shape (batch_size, src_seq_len, d_model)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return x

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个head的维度

        # 定义用于计算query, key, value的线性层
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.W_O = nn.Linear(d_model, d_model)  # 输出线性层

    def forward(self, Q, K, V, mask=None):
        """
        计算自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)  # 用于屏蔽padding和未来信息
        Returns:
            attn_output: 注意力输出, shape (batch_size, seq_len, d_model)
        """
        batch_size = Q.size(0)
        seq_len = Q.size(1)

        # 1. 线性变换得到Q, K, V
        Q = self.W_Q(Q)  # (batch_size, seq_len, d_model)
        K = self.W_K(K)  # (batch_size, seq_len, d_model)
        V = self.W_V(V)  # (batch_size, seq_len, d_model)

        # 2. 将Q, K, V按head分割
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        K = K.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)
        V = V.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len, d_k)

        # 3. 计算注意力分数
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))  # (batch_size, n_heads, seq_len, seq_len)

        # 4. 应用mask
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)  # 将mask为0的位置的score设为负无穷

        # 5. 计算注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch_size, n_heads, seq_len, seq_len)

        # 6. 计算加权后的V
        attn_output = torch.matmul(attn_weights, V)  # (batch_size, n_heads, seq_len, d_k)

        # 7. 拼接多头结果
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)  # (batch_size, seq_len, d_model)

        # 8. 通过线性层得到最终输出
        attn_output = self.W_O(attn_output)  # (batch_size, seq_len, d_model)

        return attn_output

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.self_attention = SelfAttention(d_model, n_heads)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, mask=None):
        """
        计算多头自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 多头自注意力的输出, shape (batch_size, seq_len, d_model)
        """
        # 计算自注意力
        attn_output = self.self_attention(Q, K, V, mask)

        # dropout + 残差连接 + layer norm
        output = self.dropout(attn_output) + Q
        output = self.layer_norm(output)

        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        前馈神经网络

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        # 线性层 + ReLU + 线性层
        output = self.linear_2(self.relu(self.linear_1(x)))
        output = self.dropout(output) + x  # dropout + 残差连接
        output = self.layer_norm(output)  # layer norm
        return output

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, mask=None):
        """
        编码器层

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        output = self.multi_head_attention(x, x, x, mask)
        output = self.feed_forward(output)
        return output

class Encoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, mask=None):
        """
        编码器

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, mask)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(d_model, n_heads)  # Masked Self Attention
        self.encoder_decoder_attention = MultiHeadAttention(d_model, n_heads)  # Encoder-Decoder Attention
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        解码器层

        Args:
            x: 输入, shape (batch_size, tgt_seq_len, d_model)
            encoder_output: 编码器的输出, shape (batch_size, src_seq_len, d_model)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, d_model)
        """
        # Masked self attention
        output = self.masked_multi_head_attention(x, x, x, tgt_mask)

        # Encoder-decoder attention
        output = self.encoder_decoder_attention(output, encoder_output, encoder_output, src_mask)

        # Feed forward
        output = self.feed_forward(output)
        return output

class Decoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        解码器

        Args:
            x: 输入, shape (batch_size, tgt_seq_len, d_model)
            encoder_output: 编码器的输出, shape (batch_size, src_seq_len, d_model)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return x

# 1. 定义超参数
n_layers = 6
d_model = 512
n_heads = 8
d_ff = 2048
batch_size = 32
src_seq_len = 64
tgt_seq_len = 64

# 2. 创建 Encoder 和 Decoder 实例
encoder = Encoder(n_layers, d_model, n_heads, d_ff)
decoder = Decoder(n_layers, d_model, n_heads, d_ff)

# 3. 创建随机输入张量
# 假设我们有形状为 (batch_size, src_seq_len, d_model) 的源序列输入和形状为 (batch_size, tgt_seq_len, d_model) 的目标序列输入。
encoder_input_tensor = torch.randn(batch_size, src_seq_len, d_model)
decoder_input_tensor = torch.randn(batch_size, tgt_seq_len, d_model)

# 4. 创建随机 mask
src_mask = torch.randint(0, 2, (batch_size, 1, 1, src_seq_len)).bool()
tgt_mask = torch.randint(0, 2, (batch_size, 1, tgt_seq_len, tgt_seq_len)).bool()

# 5. 将输入张量传递给 Encoder 和 Decoder 实例
encoder_output_tensor = encoder(encoder_input_tensor, src_mask)
decoder_output_tensor = decoder(decoder_input_tensor, encoder_output_tensor, src_mask, tgt_mask)

# 6. 打印输入和输出的形状
print("Encoder Input tensor shape:", encoder_input_tensor.shape)
print("Encoder Output tensor shape:", encoder_output_tensor.shape)
print("Decoder Input tensor shape:", decoder_input_tensor.shape)
print("Decoder Output tensor shape:", decoder_output_tensor.shape)
print("Source Mask shape:", src_mask.shape)
print("Target Mask shape:", tgt_mask.shape)

# 7. 打印输入张量的内容
print("Encoder Input tensor:", encoder_input_tensor)
print("Encoder Output tensor:", encoder_output_tensor)
print("Decoder Input tensor:", decoder_input_tensor)
print("Decoder Output tensor:", decoder_output_tensor)
print("Source Mask:", src_mask)
print("Target Mask:", tgt_mask)


Encoder Input tensor shape: torch.Size([32, 64, 512])
Encoder Output tensor shape: torch.Size([32, 64, 512])
Decoder Input tensor shape: torch.Size([32, 64, 512])
Decoder Output tensor shape: torch.Size([32, 64, 512])
Source Mask shape: torch.Size([32, 1, 1, 64])
Target Mask shape: torch.Size([32, 1, 64, 64])
Encoder Input tensor: tensor([[[-3.7422e-02,  4.8560e-01, -1.9716e-01,  ...,  1.7757e+00,
          -1.1385e-01,  6.8083e-02],
         [ 9.9102e-01,  2.3063e-01, -2.3128e+00,  ..., -1.0813e-01,
           1.2448e+00,  1.0214e+00],
         [-1.8463e+00,  6.8184e-01,  7.4153e-01,  ...,  2.0625e+00,
           3.7722e-01,  5.8482e-01],
         ...,
         [-3.3154e-01, -4.4327e-01, -3.0454e-01,  ...,  1.0890e-01,
           1.6019e+00, -3.8015e-02],
         [-1.2105e+00,  7.9613e-01, -6.6962e-01,  ..., -5.8293e-01,
          -1.8930e+00, -1.2643e+00],
         [-8.1272e-01,  5.7420e-02, -1.1916e+00,  ..., -9.7657e-01,
          -7.3459e-01, -2.4755e+00]],

        [[-1.3749e+00

### **Decoder：像“作家创作”一样逐步生成**

---

#### **1. 直观比喻：写作的三步法**
想象一位作家在创作小说：
1. **构思大纲（Masked Self-Attention）**：先想好已写部分的关系（不能偷看未写内容）
   - *"从前有个__" → 只能基于"从前有个"思考，不能提前想空白处的内容*
   
2. **参考素材（Encoder-Decoder Attention）**：查阅历史资料（Encoder理解的原作）
   - *把中文句子"我爱机器学习"的语义特征作为参考*
   
3. **润色段落（FeedForward）**：深化当前句子的表达
   - *把"模型"优化为"深度学习模型"*

---

#### **2. 代码拆解：创作流水线**
```python
class DecoderLayer(nn.Module):
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # 第一步：构思已写内容（屏蔽未来信息）
        x = self.masked_multi_head_attention(x, x, x, tgt_mask)
        
        # 第二步：参考素材库（编码器的理解）
        x = self.encoder_decoder_attention(x, encoder_output, encoder_output, src_mask)
        
        # 第三步：精炼当前表达
        x = self.feed_forward(x)
        return x
```

---

#### **3. Decoder的三大核心设计**
| 设计                | 作用                          | 类比                      |
|---------------------|-----------------------------|-------------------------|
| **Masked Self-Attention** | 防止偷看未来信息，保证自回归生成      | 写作文时用纸遮住后面的空白行       |
| **Encoder-Decoder Attention** | 让生成内容与源语义对齐          | 翻译时随时对照原文             |
| **渐进生成机制**      | 逐词生成，每个词依赖之前所有输出     | 接龙游戏：必须基于前文续写        |

---

#### **4. 逐层工作原理示例**
**任务**：把"I love machine learning"翻译成"我 爱 机器 学习"

- **第1个解码步（生成"我"）**：
  ```python
  # Masked Self-Attention：只能看到起始符<start>
  # Encoder-Decoder Attention：关注英文中的"I"
  ```

- **第3个解码步（生成"机器"）**：
  ```python
  # Masked Self-Attention：能看到"我 爱"
  # Encoder-Decoder Attention：聚焦"machine"
  ```

- **最终步（生成"学习"）**：
  ```python
  # Masked Self-Attention：已生成"我 爱 机器"
  # Encoder-Decoder Attention：关联"learning"
  ```

---

#### **5. 关键机制详解**
##### **5.1 Masked Self-Attention：防作弊面具**
- **实现方式**：在注意力分数矩阵上加下三角掩码
  ```python
  # 假设序列长度为3，掩码矩阵为：
  [[1, 0, 0],
   [1, 1, 0],
   [1, 1, 1]]
  ```
- **效果**：生成第2个词时，只能关注第1个词

##### **5.2 Encoder-Decoder Attention：跨语言桥**
- **Query来自Decoder**：当前生成位置的需求
- **Key/Value来自Encoder**：源语言的知识库
- **类比**：学生（Decoder）带着问题（Query），去课本（Encoder）里找答案

---

#### **6. 与Encoder的对比**
|                  | Encoder                      | Decoder                |
|------------------|-----------------------------|------------------------|
| **注意力类型**    | 全连接自注意力                 | Masked自注意力+交叉注意力  |
| **输入依赖**      | 只处理源序列                   | 依赖源序列和目标序列         |
| **运行方式**      | 一次完成全部计算                | 自回归逐词生成             |
| **典型应用**      | 理解/分类任务                 | 生成/翻译任务             |

---

#### **7. 可视化理解：瀑布式创作**
想象Decoder的工作像瀑布流水：
1. **顶层水流（第1层）**：粗糙的语义流向（确定要表达"喜爱"）
2. **中层瀑布（第3层）**：细化表达方式（选择动词"爱"）
3. **底层水池（第6层）**：精准用词（确定名词"机器学习"）

每一层都在前一层的基础上加工，最终形成完整的语义瀑布。

---

### **终极总结**
Decoder的设计蕴含三大智慧：
1. **自律性**：通过Mask机制约束自己只关注已生成内容
2. **开放性**：通过交叉注意力吸收外部知识（Encoder输出）
3. **渐进性**：像画家作画般层层叠加细节

正是这种既自律又开放的机制，使得Transformer能够完成：
- *流畅的机器翻译*
- *连贯的文本生成*
- *精准的问答输出*

就像一位严谨的作家，既要遵循创作纪律，又要广泛吸收素材，最终写出精彩篇章！ 📖

10. Transformer模型

    最后，我们将所有组件组合起来，构建完整的Transformer模型。


In [18]:

import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, n_layers, n_heads, d_ff, max_len):
        super().__init__()
        self.encoder_input_embedding = InputEmbeddings(d_model, src_vocab_size)
        self.decoder_input_embedding = InputEmbeddings(d_model, tgt_vocab_size)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff)
        self.projection = nn.Linear(d_model, tgt_vocab_size)  # 输出线性层，将解码器的输出映射到目标词汇表

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        Transformer模型

        Args:
            src: 源序列, shape (batch_size, src_seq_len)
            tgt: 目标序列, shape (batch_size, tgt_seq_len)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, tgt_vocab_size)
        """
        # 1. 计算源序列和目标序列的嵌入向量
        src_embedded = self.encoder_input_embedding(src)  # (batch_size, src_seq_len, d_model)
        tgt_embedded = self.decoder_input_embedding(tgt)  # (batch_size, tgt_seq_len, d_model)

        # 2. 加上位置编码
        src_embedded = self.positional_encoding(src_embedded)  # (batch_size, src_seq_len, d_model)
        tgt_embedded = self.positional_encoding(tgt_embedded)  # (batch_size, tgt_seq_len, d_model)

        # 3. 通过编码器处理源序列
        encoder_output = self.encoder(src_embedded, src_mask)  # (batch_size, src_seq_len, d_model)

        # 4. 通过解码器处理目标序列
        decoder_output = self.decoder(tgt_embedded, encoder_output, src_mask, tgt_mask)  # (batch_size, tgt_seq_len, d_model)

        # 5. 通过线性层得到最终输出
        output = self.projection(decoder_output)  # (batch_size, tgt_seq_len, tgt_vocab_size)

        return output


11. Mask的实现

    在Transformer模型中，mask用于屏蔽某些位置的信息，以防止模型在训练过程中作弊。

    有两种类型的mask：源序列mask (src_mask): 用于屏蔽源序列中的填充（padding）token。目标序列mask (tgt_mask): 用于屏蔽目标序列中未来位置的token，以确保模型在生成每个token时只能依赖于之前生成的token。



In [19]:

def create_masks(src, tgt, pad_token):
    """
    生成源序列和目标序列的mask

    Args:
        src: 源序列, shape (batch_size, src_seq_len)
        tgt: 目标序列, shape (batch_size, tgt_seq_len)
        pad_token: padding token的索引
    Returns:
        src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
        tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
    """
    batch_size = src.size(0)
    src_seq_len = src.size(1)
    tgt_seq_len = tgt.size(1)

    # 1. 创建源序列mask
    src_mask = (src != pad_token).unsqueeze(1).unsqueeze(2)  # (batch_size, 1, 1, src_seq_len)

    # 2. 创建目标序列的padding mask
    tgt_pad_mask = (tgt != pad_token).unsqueeze(1).unsqueeze(3)  # (batch_size, 1, tgt_seq_len, 1)

    # 3. 创建一个下三角矩阵，用于屏蔽未来位置的信息
    subsequent_mask = torch.tril(torch.ones((tgt_seq_len, tgt_seq_len), device=src.device)).bool()
    subsequent_mask = subsequent_mask.unsqueeze(0).unsqueeze(1)  # (1, 1, tgt_seq_len, tgt_seq_len)

    # 4. 组合padding mask和subsequent mask
    tgt_mask = tgt_pad_mask & subsequent_mask  # (batch_size, 1, tgt_seq_len, tgt_seq_len)

    return src_mask, tgt_mask



12. 训练Transformer模型

    现在，我们可以使用上述组件来训练Transformer模型。我们将使用一个简单的序列到序列的任务，例如将一种语言翻译成另一种语言。



In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # 添加缺失的导入
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class InputEmbeddings(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        # 将词嵌入向量乘以根号d_model，以缩小数值范围，这有助于在训练过程中稳定梯度。
        return self.embedding(x) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        # 创建一个形状为 (max_len, d_model) 的矩阵，用于保存位置编码
        pe = torch.zeros(max_len, d_model)
        # 创建一个形状为 (max_len, 1) 的向量，包含位置信息
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        # 计算位置除以 (10000 ** (2i / d_model)) 的值，用于后续计算
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        # 计算偶数位置的正弦值
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算奇数位置的余弦值
        pe[:, 1::2] = torch.cos(position * div_term)
        # 将 pe 的形状变为 (1, max_len, d_model)，以便与嵌入向量相加
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 将输入 x 的词嵌入向量与对应位置的位置编码相加
        # x.shape: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :]

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个head的维度

        # 定义用于计算query, key, value的线性层
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.W_O = nn.Linear(d_model, d_model)  # 输出线性层

    def forward(self, Q, K, V, mask=None):
        """
        计算自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)  # 用于屏蔽padding和未来信息
        Returns:
            attn_output: 注意力输出, shape (batch_size, seq_len, d_model)
        """
        batch_size = Q.size(0)
        seq_len_q = Q.size(1)
        seq_len_k = K.size(1)
        seq_len_v = V.size(1)

        # 1. 线性变换得到Q, K, V
        Q = self.W_Q(Q)  # (batch_size, seq_len_q, d_model)
        K = self.W_K(K)  # (batch_size, seq_len_k, d_model)
        V = self.W_V(V)  # (batch_size, seq_len_v, d_model)

        # 2. 将Q, K, V按head分割
        Q = Q.view(batch_size, seq_len_q, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len_q, d_k)
        K = K.view(batch_size, seq_len_k, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len_k, d_k)
        V = V.view(batch_size, seq_len_v, self.n_heads, self.d_k).transpose(1, 2)  # (batch_size, n_heads, seq_len_v, d_k)

        # 3. 计算注意力分数
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))  # (batch_size, n_heads, seq_len_q, seq_len_k)

        # 4. 应用mask
        if mask is not None:
            # 确保mask与注意力分数的形状匹配
            if mask.size(-1) != attn_scores.size(-1) or mask.size(-2) != attn_scores.size(-2):
                # 调整mask大小以匹配注意力分数
                if len(mask.size()) == 3:  # 如果mask是3D的(batch_size, 1, seq_len)
                    mask = mask.unsqueeze(2)  # 变为(batch_size, 1, 1, seq_len)
                mask = mask[:, :, :attn_scores.size(2), :attn_scores.size(3)]
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)  # 将mask为0的位置的score设为负无穷

        # 5. 计算注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch_size, n_heads, seq_len_q, seq_len_k)

        # 6. 计算加权后的V
        attn_output = torch.matmul(attn_weights, V)  # (batch_size, n_heads, seq_len_q, d_k)

        # 7. 拼接多头结果
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len_q, self.d_model)  # (batch_size, seq_len_q, d_model)

        # 8. 通过线性层得到最终输出
        attn_output = self.W_O(attn_output)  # (batch_size, seq_len_q, d_model)

        return attn_output

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.self_attention = SelfAttention(d_model, n_heads)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, mask=None):
        """
        计算多头自注意力

        Args:
            Q: query, shape (batch_size, seq_len, d_model)
            K: key, shape (batch_size, seq_len, d_model)
            V: value, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 多头自注意力的输出, shape (batch_size, seq_len, d_model)
        """
        # 计算自注意力
        attn_output = self.self_attention(Q, K, V, mask)

        # dropout + 残差连接 + layer norm
        output = self.dropout(attn_output) + Q
        output = self.layer_norm(output)

        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        前馈神经网络

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        # 线性层 + ReLU + 线性层
        output = self.linear_2(self.relu(self.linear_1(x)))
        output = self.dropout(output) + x  # dropout + 残差连接
        output = self.layer_norm(output)  # layer norm
        return output

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, mask=None):
        """
        编码器层

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        output = self.multi_head_attention(x, x, x, mask)
        output = self.feed_forward(output)
        return output

class Encoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, mask=None):
        """
        编码器

        Args:
            x: 输入, shape (batch_size, seq_len, d_model)
            mask: mask, shape (batch_size, 1, seq_len, seq_len)
        Returns:
            output: 输出, shape (batch_size, seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, mask)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(d_model, n_heads)
        self.encoder_decoder_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        解码器层

        Args:
            x: 输入, shape (batch_size, tgt_seq_len, d_model)
            encoder_output: 编码器的输出, shape (batch_size, src_seq_len, d_model)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, d_model)
        """
        # Masked self attention
        output = self.masked_multi_head_attention(x, x, x, tgt_mask)

        # Encoder-decoder attention
        output = self.encoder_decoder_attention(output, encoder_output, encoder_output, src_mask)

        # Feed forward
        output = self.feed_forward(output)
        return output

class Decoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff) for _ in range(n_layers)])

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        解码器

        Args:
            x: 输入, shape (batch_size, tgt_seq_len, d_model)
            encoder_output: 编码器的输出, shape (batch_size, src_seq_len, d_model)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, n_layers, n_heads, d_ff, max_len):
        super().__init__()
        self.encoder_input_embedding = InputEmbeddings(d_model, src_vocab_size)
        self.decoder_input_embedding = InputEmbeddings(d_model, tgt_vocab_size)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff)
        self.projection = nn.Linear(d_model, tgt_vocab_size)  # 输出线性层，将解码器的输出映射到目标词汇表

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        Transformer模型

        Args:
            src: 源序列, shape (batch_size, src_seq_len)
            tgt: 目标序列, shape (batch_size, tgt_seq_len)
            src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
            tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
        Returns:
            output: 输出, shape (batch_size, tgt_seq_len, tgt_vocab_size)
        """
        # 1. 计算源序列和目标序列的嵌入向量
        src_embedded = self.encoder_input_embedding(src)  # (batch_size, src_seq_len, d_model)
        tgt_embedded = self.decoder_input_embedding(tgt)  # (batch_size, tgt_seq_len, d_model)

        # 2. 加上位置编码
        src_embedded = self.positional_encoding(src_embedded)  # (batch_size, src_seq_len, d_model)
        tgt_embedded = self.positional_encoding(tgt_embedded)  # (batch_size, tgt_seq_len, d_model)

        # 3. 通过编码器处理源序列
        encoder_output = self.encoder(src_embedded, src_mask)  # (batch_size, src_seq_len, d_model)

        # 4. 通过解码器处理目标序列
        decoder_output = self.decoder(tgt_embedded, encoder_output, src_mask, tgt_mask)  # (batch_size, tgt_seq_len, d_model)

        # 5. 通过线性层得到最终输出
        output = self.projection(decoder_output)  # (batch_size, tgt_seq_len, tgt_vocab_size)

        return output

def create_masks(src, tgt, pad_token):
    """
    生成源序列和目标序列的mask

    Args:
        src: 源序列, shape (batch_size, src_seq_len)
        tgt: 目标序列, shape (batch_size, tgt_seq_len)
        pad_token: padding token的索引
    Returns:
        src_mask: 源序列的mask, shape (batch_size, 1, 1, src_seq_len)
        tgt_mask: 目标序列的mask, shape (batch_size, 1, tgt_seq_len, tgt_seq_len)
    """
    batch_size = src.size(0)
    src_seq_len = src.size(1)
    tgt_seq_len = tgt.size(1)

    # 1. 创建源序列mask
    src_mask = (src != pad_token).unsqueeze(1).unsqueeze(2)  # (batch_size, 1, 1, src_seq_len)

    # 2. 创建目标序列的padding mask
    tgt_pad_mask = (tgt != pad_token).unsqueeze(1).unsqueeze(3)  # (batch_size, 1, tgt_seq_len, 1)

    # 3. 创建一个下三角矩阵，用于屏蔽未来位置的信息
    subsequent_mask = torch.tril(torch.ones((tgt_seq_len, tgt_seq_len), device=src.device)).bool()
    subsequent_mask = subsequent_mask.unsqueeze(0).unsqueeze(1)  # (1, 1, tgt_seq_len, tgt_seq_len)

    # 4. 组合padding mask和subsequent mask
    tgt_mask = tgt_pad_mask & subsequent_mask  # (batch_size, 1, tgt_seq_len, tgt_seq_len)

    return src_mask, tgt_mask

# 1. 定义超参数
src_vocab_size = 10000
tgt_vocab_size = 10000
d_model = 512
n_layers = 6
n_heads = 8
d_ff = 2048
max_len = 100
pad_token = 0  # 假设0为padding token
batch_size = 32
epochs = 10
learning_rate = 0.0001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 创建Transformer模型
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, n_layers, n_heads, d_ff, max_len).to(device)

# 3. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss(ignore_index=pad_token)  # 忽略padding token的损失
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 4. 创建虚拟数据
src_data = torch.randint(1, src_vocab_size, (1000, max_len)).to(device)  # (1000, max_len)
tgt_data = torch.randint(1, tgt_vocab_size, (1000, max_len)).to(device)  # (1000, max_len)
dataset = TensorDataset(src_data, tgt_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 5. 训练模型
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for i, (src, tgt) in enumerate(dataloader):
        # 准备数据
        src = src.to(device)  # (batch_size, src_seq_len)
        tgt = tgt.to(device)  # (batch_size, tgt_seq_len)

        # 教师强制：使用tgt的前面部分(不包括最后一个token)作为解码器输入
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]  # 预测目标：tgt的后面部分(不包括第一个token)

        # 创建mask
        src_mask, tgt_mask = create_masks(src, tgt_input, pad_token)

        # 前向传播
        output = model(src, tgt_input, src_mask, tgt_mask)  # (batch_size, tgt_seq_len-1, tgt_vocab_size)

        # 计算损失
        output = output.reshape(-1, tgt_vocab_size)  # (batch_size * (tgt_seq_len-1), tgt_vocab_size)
        tgt_output = tgt_output.reshape(-1)  # (batch_size * (tgt_seq_len-1))
        loss = criterion(output, tgt_output)
        total_loss += loss.item()

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()

        # 梯度裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        # 打印损失
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    # 打印每个epoch的平均损失
    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}')

print('Training finished!')

Epoch [1/10], Step [10/32], Loss: 9.3607
Epoch [1/10], Step [20/32], Loss: 9.3210
Epoch [1/10], Step [30/32], Loss: 9.3058
Epoch [1/10], Average Loss: 9.3341
Epoch [2/10], Step [10/32], Loss: 9.2285
Epoch [2/10], Step [20/32], Loss: 9.2097
Epoch [2/10], Step [30/32], Loss: 9.2260
Epoch [2/10], Average Loss: 9.2215
Epoch [3/10], Step [10/32], Loss: 9.1852
Epoch [3/10], Step [20/32], Loss: 9.1984
Epoch [3/10], Step [30/32], Loss: 9.2015
Epoch [3/10], Average Loss: 9.1890
Epoch [4/10], Step [10/32], Loss: 9.1524
Epoch [4/10], Step [20/32], Loss: 9.1513
Epoch [4/10], Step [30/32], Loss: 9.1694
Epoch [4/10], Average Loss: 9.1499
Epoch [5/10], Step [10/32], Loss: 8.9823
Epoch [5/10], Step [20/32], Loss: 8.9682
Epoch [5/10], Step [30/32], Loss: 8.9298
Epoch [5/10], Average Loss: 9.0002
Epoch [6/10], Step [10/32], Loss: 8.7823
Epoch [6/10], Step [20/32], Loss: 8.7316
Epoch [6/10], Step [30/32], Loss: 8.7828
Epoch [6/10], Average Loss: 8.7828
Epoch [7/10], Step [10/32], Loss: 8.5778
Epoch [7/10